In [7]:
from Module.MySQL_connector import MySQLConnector
import pandas as pd
import os

In [3]:
sql_connector = MySQLConnector('192.168.31.130', 'bigred', 'bigred', 32010)
query = 'RENAME TABLE first30days_sales_2A TO first30days_sales_aa;'
sql_connector.connectDB('g3_db')
sql_connector.execute_query(query)

NameError: name 'MySQLConnector' is not defined

In [5]:
def data_fe_pipline(pub_class): 
# import data from MySQL
    sql_connector = MySQLConnector('192.168.31.130', 'bigred', 'bigred', 32010)
    sql_connector.connectDB('g3_db')
    # print(sql_connector.execute_query('show tables;'))
    # sql_connector.execute_query('show tables;')
    main_table = pd.DataFrame(sql_connector.execute_query(f"select * from main_{pub_class}"))
    print('mysql done')
    genres_onehot = pd.DataFrame(sql_connector.execute_query(f"select * from genres_onehot_{pub_class}"))
    features_onehot = pd.DataFrame(sql_connector.execute_query(f"select * from features_onehot_{pub_class}"))
    tags_onehot = pd.DataFrame(sql_connector.execute_query(f"select * from tags_onehot_{pub_class}"))
    id_publisher_aa = pd.DataFrame(sql_connector.execute_query(f"select * from id_publishers_{pub_class}"))
    publishers_info_aa = pd.DataFrame(sql_connector.execute_query(f"select * from publishers_info_{pub_class};"))
    print('mysql done')
    if pub_class == 'aa':
        copiesold = pd.DataFrame(sql_connector.execute_query("select * from first30days_sales_2A")).drop(columns=['timeStamp','releaseDate','days_since_release'])
    elif pub_class == 'aaa':
        copiesold = pd.DataFrame(sql_connector.execute_query("select * from first30days_sales_3A")).drop(columns=['timeStamp','releaseDate','days_since_release'])
    else:
        copiesold = pd.DataFrame(sql_connector.execute_query("select * from first30days_sales_indie")).drop(columns=['timeStamp','releaseDate','days_since_release'])
    print('mysql done')
    merged_table = pd.merge(main_table, genres_onehot, on='steamId', suffixes=('', '_genres'))
    merged_table = pd.merge(merged_table, tags_onehot, on='steamId', suffixes=('', '_tags'))
    # 再次合併時，處理更多的重複列
    merged_table = pd.merge(merged_table, features_onehot, on='steamId', suffixes=('', '_features'))
    # 找到所有帶有 '_tags' 或 '_features' 後綴的列名
    replicated_columns = [col for col in merged_table.columns if '_tags' in col or '_features' in col]

    # 刪除這些列
    merged_table.drop(columns=replicated_columns, inplace=True)
    cs_pub = pd.merge(copiesold,id_publisher_aa,on="steamId")
    # cs_pub_info = pd.merge(cs_pub, publishers_info_aa, on="steamId")
    cs_pub_info = pd.merge(cs_pub, publishers_info_aa, left_on='publisher', right_on="name").dropna()
    full_table = pd.merge(cs_pub_info, merged_table, on="steamId",suffixes=('', '_pub'))
    columns_to_drop = ['steamId','totalRevenue_pub','publisher','name_pub','name','reviews','reviewsSteam','followers','avgPlaytime','reviewScore','releaseDate','firstReleaseDate','copiesSold','revenue','totalRevenue','players','owners','wishlists']
    full_table=full_table.drop(columns=columns_to_drop).dropna()
    return full_table

In [5]:
# 合併時指定 suffixes 參數來處理重複列
data_fe_pipline('aa')

In [11]:

full_table

KeyError: "['steamId', 'totalRevenue_pub', 'publisher', 'name_pub', 'name', 'reviews', 'reviewsSteam', 'followers', 'avgPlaytime', 'reviewScore', 'releaseDate', 'firstReleaseDate', 'copiesSold', 'revenue', 'totalRevenue', 'players', 'owners', 'wishlists'] not found in axis"

,sales,averageRevenue,medianRevenue,publishedGames,price,earlyAccess,Action,Adventure,Casual,Early Access,...,Steam Cloud,Steam Leaderboards,Steam Trading Cards,Steam Turn Notifications,Steam Workshop,SteamVR Collectibles,Tracked Controller Support,VR Only,VR Supported,Valve Anti-Cheat enabled
0,0.000000e+00,3.097318e+06,555213.0,127,5.99,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,5.691300e+04,3.097318e+06,555213.0,127,5.99,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1.796316e+05,2.301824e+07,21042950.0,8,19.99,0,1,1,0,0,...,1,1,1,0,0,0,0,0,0,0
3,1.099500e+06,1.259276e+07,2115005.0,11,39.99,0,0,0,0,0,...,1,0,1,0,1,0,0,0,0,0
4,5.393900e+04,6.568174e+06,2458665.0,35,19.99,0,1,1,0,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1239,1.050977e+06,8.568731e+06,5725398.0,7,7.99,0,1,1,0,0,...,1,0,0,0,0,0,0,0,0,0
1240,4.581000e+03,1.565075e+06,425891.5,50,59.99,0,1,1,0,0,...,1,0,1,0,0,0,0,0,0,0
1241,9.695800e+04,3.749863e+06,708029.0,49,19.99,0,1,0,0,0,...,1,0,1,0,0,0,0,0,0,0
1242,2.590000e+03,1.565075e+06,425891.5,50,39.99,0,1,1,0,0,...,1,0,1,0,0,0,0,0,0,0
